In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import ast

In [2]:
credentials = service_account.Credentials.from_service_account_file(
    'C:/Users/pfbab/NSS/googlekey/solar-roof-capstone-787ff1e58b5f.json',
)
client = bigquery.Client(credentials=credentials)

In [3]:
query = """SELECT solar.*, geo_id, do_date, total_pop, households, median_age, median_income, income_per_capita,
         poverty, housing_units, renter_occupied_housing_units_paying_cash_median_gross_rent,
          owner_occupied_housing_units_lower_value_quartile, owner_occupied_housing_units_median_value,
           owner_occupied_housing_units_upper_value_quartile, occupied_housing_units, housing_units_renter_occupied,
            vacant_housing_units, vacant_housing_units_for_rent, vacant_housing_units_for_sale, housing_built_2005_or_later,
             housing_built_2000_to_2004, housing_built_1939_or_earlier, median_year_structure_built, median_rent,
              percent_income_spent_on_rent, owner_occupied_housing_units, mortgaged_housing_units, 
              pop_in_labor_force, not_in_labor_force 
FROM `bigquery-public-data.sunroof_solar.solar_potential_by_censustract` AS solar
LEFT JOIN `solar-roof-capstone.capdatasets.ACS18_for_roofdata` AS census
ON   solar.region_name = census.geo_id
WHERE yearly_sunlight_kwh_total > 0"""

In [4]:
query_job = client.query(query)

In [5]:
capmerge = query_job.to_dataframe()

In [6]:
capmerge

,region_name,state_name,lat_max,lat_min,lng_max,lng_min,lat_avg,lng_avg,yearly_sunlight_kwh_kw_threshold_avg,count_qualified,...,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,median_rent,percent_income_spent_on_rent,owner_occupied_housing_units,mortgaged_housing_units,pop_in_labor_force,not_in_labor_force
0,13115000201,Georgia,34.371120,34.287239,-85.117592,-85.231651,34.330746,-85.169342,1055.70,1,...,94.0,34.0,88.0,1993.0,539.0,30.9,1426.0,1020.0,3197.0,1848.0
1,18057110512,Indiana,40.101791,40.076389,-86.038544,-86.090332,40.092869,-86.060478,983.45,1,...,0.0,0.0,46.0,1986.0,740.0,24.2,1313.0,996.0,2791.0,937.0
2,32031980200,Nevada,39.635529,39.548290,-119.845078,-120.001953,39.588264,-119.929153,1315.80,2,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,01055010700,Alabama,34.152290,33.987690,-85.822540,-85.987312,34.065083,-85.893082,1061.65,1,...,0.0,40.0,33.0,1987.0,491.0,29.7,1163.0,535.0,1826.0,1284.0
4,42043024700,Pennsylvania,40.571880,40.336391,-76.535408,-77.027008,40.425526,-76.812088,987.70,2,...,68.0,34.0,122.0,1968.0,635.0,21.2,2099.0,1164.0,3499.0,1567.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52564,26065004491,Michigan,42.732880,42.728729,-84.472054,-84.475311,42.730747,-84.473511,937.55,4,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,531.0,697.0
52565,48041002015,Texas,30.627750,30.604610,-96.323433,-96.348572,30.616558,-96.336372,1088.00,162,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1573.0,7763.0
52566,12011100104,Florida,25.986570,25.975019,-80.119270,-80.123306,25.980501,-80.121162,1135.60,52,...,0.0,27.0,24.0,1974.0,1426.0,26.5,1061.0,334.0,1051.0,1386.0
52567,51107611020,Virginia,39.024670,38.978981,-77.428917,-77.506088,39.001179,-77.462639,1056.55,206,...,237.0,533.0,26.0,2004.0,1654.0,25.0,1256.0,1119.0,4235.0,1043.0


In [7]:
def lister(row):
    try:
        res = ast.literal_eval(row.install_size_kw_buckets)
    except:
        res = [[0,0]]
    return res

In [8]:
capmerge['install_size_kw_buckets'] = capmerge.apply(lambda x: lister(x), axis=1)

In [9]:
buckets = pd.DataFrame()
for index, row in capmerge.iterrows():
    bucket = {}
    bucket['region_name'] = row['region_name']
    bucket['install_size_kw_bucket200+'] = 0
    bucket['install_size_kw_bucket150-200'] = 0
    bucket['install_size_kw_bucket100-149'] = 0
    bucket['install_size_kw_bucket50-99'] = 0
    bucket['install_size_kw_bucket15-49'] = 0
    a, b, c, d, e = 0,0,0,0,0
    for buc in row.install_size_kw_buckets:
        if buc[0] > 200:
            bucket['install_size_kw_bucket200+'] += buc[1]
            a += buc[0] * buc[1]
            bucket['install_size_kw_bucket200+_avg'] = a / bucket['install_size_kw_bucket200+']
        elif buc[0] > 150:
            bucket['install_size_kw_bucket150-200'] += buc[1]
            b += buc[0] * buc[1]
            bucket['install_size_kw_bucket150-200_avg'] = b / bucket['install_size_kw_bucket150-200']
        elif buc[0] > 100:
            bucket['install_size_kw_bucket100-149'] += buc[1]
            c += buc[0] * buc[1]
            bucket['install_size_kw_bucket100-149_avg'] = c / bucket['install_size_kw_bucket100-149']
        elif buc[0] > 50:
            bucket['install_size_kw_bucket50-99'] += buc[1]  
            d += buc[0] * buc[1]
            bucket['install_size_kw_bucket50-99_avg'] = d / bucket['install_size_kw_bucket50-99']
        elif buc[0] >= 15:
            bucket['install_size_kw_bucket15-49'] += buc[1]  
            e += buc[0] * buc[1]
            bucket['install_size_kw_bucket15-49_avg'] = e / bucket['install_size_kw_bucket15-49']
        else:
            bucket['install_size_kw_bucket' + str(buc[0])] = buc[1]
            
    
    buckets = buckets.append(bucket, ignore_index=True)

In [10]:
for col in buckets.columns:
    capmerge[col] = buckets[col]

In [11]:
capmerge = capmerge.sort_values(by='number_of_panels_total').drop_duplicates(subset='region_name', keep='last')

In [12]:
capmerge.to_csv('../data/capmerge.csv', index=False)